<a href="https://colab.research.google.com/github/KhushiiAgarwal/AI/blob/main/Azure_openAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import json
from tabulate import tabulate

In [ ]:
def process_json_tables(json_file=None, json_data=None):
    """
    Process all tables from a JSON file or string containing table data.

    Parameters:
    json_file (str, optional): Path to JSON file
    json_data (str, optional): JSON string data

    Returns:
    dict: Dictionary of formatted tables and metadata
    """
    # Load the data
    if json_file:
        with open(json_file, 'r') as f:
            data = json.load(f)
    elif json_data:
        data = json.loads(json_data)
    else:
        raise ValueError("Either json_file or json_data must be provided")

    # Check if 'tables' key exists
    if 'tables' not in data:
        raise KeyError("The JSON data does not contain a 'tables' key")

    # Extract and process all tables
    result = {}
    tables = data['tables']
    result['table_count'] = len(tables)

    # Dictionary to store formatted tables
    formatted_tables = {}
    table_metadata = {}

    print(f"Found {len(tables)} tables in the document.")

    # Process each table
    for i, table in enumerate(tables):
        table_id = table.get('table_id', f'table_{i}')

        # Basic formatting
        df = format_table_from_json(table)

        # Advanced formatting with header info
        df_with_headers, header_positions = format_table_with_headers(table)

        # Store results
        formatted_tables[table_id] = {
            'basic': df,
            'with_headers': df_with_headers
        }

        table_metadata[table_id] = {
            'row_count': table['row_count'],
            'column_count': table['column_count'],
            'cell_count': len(table['cells']),
            'header_positions': list(header_positions)
        }

        # Print basic info about the table
        print(f"\nTable {i+3}: {table_id}")
        print(f"Dimensions: {table['row_count']} rows x {table['column_count']} columns")
        print(f"Cell count: {len(table['cells'])}")
        print(f"Has headers: {len(header_positions) > 0}")

    # Store all processed data
    result['formatted_tables'] = formatted_tables
    result['table_metadata'] = table_metadata

    return result


In [ ]:
def format_table_from_json(table_data):
    """
    Format table data from a JSON structure into a pandas DataFrame.

    Parameters:
    table_data (dict): JSON structure containing table information

    Returns:
    pandas.DataFrame: Formatted table
    """
    # Extract row and column counts
    row_count = table_data["row_count"]
    column_count = table_data["column_count"]

    # Create an empty DataFrame with the right dimensions
    df = pd.DataFrame(np.empty((row_count, column_count), dtype=object))

    # Fill the DataFrame with content from the cells
    for cell in table_data["cells"]:
        row_idx = cell["row_index"]
        col_idx = cell["column_index"]
        content = cell["content"]
        df.iloc[row_idx, col_idx] = content

    # Replace None/NaN values with empty strings
    df = df.fillna('')

    return df


In [ ]:
def format_table_with_headers(table_data):
    """
    Format table data with header information from a JSON structure into a pandas DataFrame.

    Parameters:
    table_data (dict): JSON structure containing table information with 'kind' field

    Returns:
    tuple: (DataFrame, header_positions) where header_positions is a set of (row, col) tuples
    """
    # Extract row and column counts
    row_count = table_data["row_count"]
    column_count = table_data["column_count"]

    # Create an empty DataFrame with the right dimensions
    df = pd.DataFrame(np.empty((row_count, column_count), dtype=object))

    # Track header positions
    header_positions = set()

    # Fill the DataFrame with content from the cells
    for cell in table_data["cells"]:
        row_idx = cell["row_index"]
        col_idx = cell["column_index"]
        content = cell["content"]

        # Store the cell content
        df.iloc[row_idx, col_idx] = content

        # Check if the cell is a header (look for 'kind' field that contains 'header')
        if "kind" in cell and "header" in cell["kind"].lower():
            header_positions.add((row_idx, col_idx))

    # Replace None/NaN values with empty strings
    df = df.fillna('')

    return df, header_positions


In [ ]:
# def display_dataframe(df, header_positions=None):
#     """
#     Display a DataFrame with formatted headers.

#     Parameters:
#     df (pandas.DataFrame): The DataFrame to display
#     header_positions (set, optional): Set of (row, col) tuples indicating header positions
#     """
#     # Try to use tabulate for better console display if available
#     try:
#         if header_positions:
#             # Create a copy with bold headers for display
#             display_data = []
#             for row_idx in range(df.shape[0]):
#                 row_data = []
#                 for col_idx in range(df.shape[3]):
#                     value = df.iloc[row_idx, col_idx]
#                     if (row_idx, col_idx) in header_positions:
#                         # Mark headers with asterisks for emphasis
#                         row_data.append(f"**{value}**")
#                     else:
#                         row_data.append(value)
#                 display_data.append(row_data)

#             print(tabulate(display_data, tablefmt='grid'))
#         else:
#             print(tabulate(df, headers='keys', tablefmt='grid'))
#     except (ImportError, NameError):
#         # Fall back to standard display if tabulate is not available
#         print(df)


In [ ]:
def display_dataframe(df, header_positions=None):
    """
    Display a DataFrame with formatted headers.

    Parameters:
    df (pandas.DataFrame): The DataFrame to display
    header_positions (set, optional): Set of (row, col) tuples indicating header positions
    """
    # Try to use tabulate for better console display if available
    try:
        if header_positions:
            # Create a copy with bold headers for display
            display_data = []
            for row_idx in range(df.shape[0]):
                row_data = []
                # Use df.shape[1] to get the correct number of columns
                for col_idx in range(df.shape[1]):  # Changed from df.shape[3] to df.shape[1]
                    value = df.iloc[row_idx, col_idx]
                    if (row_idx, col_idx) in header_positions:
                        # Mark headers with asterisks for emphasis
                        row_data.append(f"**{value}**")
                    else:
                        row_data.append(value)
                display_data.append(row_data)

            print(tabulate(display_data, tablefmt='grid'))
        else:
            print(tabulate(df, headers='keys', tablefmt='grid'))
    except (ImportError, NameError):
        # Fall back to standard display if tabulate is not available
        print(df)

In [ ]:
def extract_specific_page_tables(data, page_number):
    """
    Extract tables from a specific page in the document.

    Parameters:
    data (dict): Loaded JSON data with 'tables' key
    page_number (int): Page number to extract tables from

    Returns:
    list: List of tables from the specified page
    """
    if 'tables' not in data:
        raise KeyError("The JSON data does not contain a 'tables' key")

    page_id = f"page_{page_number}"
    filtered_tables = []

    for table in data['tables']:
        if 'table_id' in table and page_id in table['table_id']:
            filtered_tables.append(table)

    return filtered_tables


In [ ]:
# Example usage:
if __name__ == "__main__":
    # Process the sample data
    data_file = r'/content/extracted_form_data_azure_form_recogn.json'  # Path to your JSON file
    with open(data_file, 'r') as f:
            data = json.load(f)
    sample_data=data
    results = process_json_tables(json_data=json.dumps(sample_data))


Found 274 tables in the document.

Table 3: page_1_table_0
Dimensions: 5 rows x 5 columns
Cell count: 17
Has headers: False

Table 4: page_1_table_1
Dimensions: 7 rows x 4 columns
Cell count: 19
Has headers: False

Table 5: page_2_table_0
Dimensions: 5 rows x 3 columns
Cell count: 12
Has headers: False

Table 6: page_2_table_1
Dimensions: 21 rows x 4 columns
Cell count: 40
Has headers: False

Table 7: page_3_table_0
Dimensions: 5 rows x 3 columns
Cell count: 12
Has headers: False

Table 8: page_3_table_1
Dimensions: 19 rows x 3 columns
Cell count: 57
Has headers: False

Table 9: page_4_table_0
Dimensions: 5 rows x 3 columns
Cell count: 12
Has headers: False

Table 10: page_4_table_1
Dimensions: 10 rows x 3 columns
Cell count: 30
Has headers: False

Table 11: page_5_table_0
Dimensions: 5 rows x 3 columns
Cell count: 12
Has headers: False

Table 12: page_5_table_1
Dimensions: 24 rows x 2 columns
Cell count: 35
Has headers: False

Table 13: page_6_table_0
Dimensions: 5 rows x 3 columns
Ce

In [ ]:

    # Display the first table
    print("\n\nDetailed view of first table:")
    first_table_id = list(results['formatted_tables'].keys())[0]
    first_table = results['formatted_tables'][first_table_id]['with_headers']
    first_table_headers = set(results['table_metadata'][first_table_id]['header_positions'])
    display_dataframe(first_table, first_table_headers)

    # Extract tables from page 31
    print("\n\nExtracted tables from page 31:")
    page_31_tables = extract_specific_page_tables(sample_data, 31)
    print(f"Found {len(page_31_tables)} tables on page 31")

    # Display each table from page 31
    for i, table in enumerate(page_31_tables):
        print(f"\nPage 31 Table {i+3}:")
        df, headers = format_table_with_headers(table)
        display_dataframe(df, headers)



Detailed view of first table:
+----+-----------------+--------------------------------------------+------------+------------+-----------+
|    | 0               | 1                                          | 2          | 3          | 4         |
+====+=================+============================================+============+============+===========+
|  0 | amneal          | BATCH MANUFACTURING RECORD                 |            |            |           |
+----+-----------------+--------------------------------------------+------------+------------+-----------+
|  1 | NAME OF PRODUCT | Multiple Electrolytes Injection Type 1 USP |            |            | BATCH NO. |
+----+-----------------+--------------------------------------------+------------+------------+-----------+
|  2 | FILLING LINE    | BAG LINE (LINE-2)                          | PAGE NO.   | I OF I     | AH230017  |
+----+-----------------+--------------------------------------------+------------+------------+---------